In [167]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.nn import functional as F

from torch.utils.data import TensorDataset, DataLoader

from tqdm.auto import trange

In [ ]:
!pip install navec
from navec import Navec
path = '../../static/embeddings/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [168]:
train_data = pd.read_csv('../../static/datasets/original/train_data.csv', sep='\t')

In [169]:
def to_emb (train_data, navec):

    def sen_split(sent, elem_split):
        for i in range(len(sent)):
            a = ((sent[i]).split(elem_split))
            if '' in a:
                a.remove('')
            if a:
                sent[i] = a[0].lower()
        return sent

    sentences = train_data.sentence
    emb = np.zeros((len(sentences), 100, 300))
    for i in range(len(sentences)):
        sentence = sentences[i].split()
        sentence = sen_split(sentence, ",")
        sentence = sen_split(sentence, ":")
        sentence = sen_split(sentence, ";")
        sentence = sen_split(sentence, ".")
        sentence = sen_split(sentence, "-")
        sentence = sen_split(sentence, "?")
        sentence = sen_split(sentence, "!")
        sentence = sen_split(sentence, ")")
        sentence = sen_split(sentence, "(")
        sentence = sen_split(sentence, '"')
        sentence = sen_split(sentence, "'")
        sentence = sen_split(sentence, "«")
        sentence = sen_split(sentence, "»")
        for j in range(len(sentence)):
            if sentence[j] in navec:
                emb[i][j] = navec[sentence[j]]
    return emb

In [171]:
X = to_emb(train_data, navec)
y = pd.DataFrame(train_data['label'])
y.replace(1, 2, inplace=True)
y.replace(0, 1, inplace=True)
y.replace(-1, 0, inplace=True)
y = np.array(y)

In [172]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

In [173]:
train_dataloader = DataLoader(TensorDataset(torch.permute(torch.from_numpy(X_train), (0, 2, 1)), torch.from_numpy(np.reshape(y_train, len(y_train)))), batch_size=128, shuffle=True)
val_dataloader = DataLoader(TensorDataset(torch.permute(torch.from_numpy(X_val), (0, 2, 1)), torch.from_numpy(np.reshape(y_val, len(y_val)))), batch_size=128, shuffle=True)

In [174]:
def fit(epochs, model_, loss_func, opt, train_dl, valid_dl, lr_sched=None):
    model_ = model_.to(device)
    train_losses = []
    val_losses = []
    valid_accuracies = []
    pbar = trange(epochs, desc="Epoch:")
    for epoch in pbar:
        model_.train()
        loss_sum = 0

        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            probs = model_(xb.float())
            loss = loss_func(probs, yb)
            loss_sum += loss.item()
            loss.backward()

            opt.step()
            opt.zero_grad()

        train_losses.append(loss_sum / len(train_dl))


        model_.eval()
        loss_sum = 0
        correct = 0
        num = 0

        with torch.no_grad():
            for xb, yb in valid_dl:
                xb, yb = xb.to(device), yb.to(device)
                probs = model_(xb.float())

                loss = loss_func(probs, yb)
                loss_sum += loss.item()

                _, pred = torch.max(probs, axis=-1)
                correct += (pred == yb).sum().item()
                num += len(xb)


        val_loss = loss_sum / len(valid_dl)
        val_acc = correct / num
        val_losses.append(val_loss)
        valid_accuracies.append(val_acc)

        pbar.set_description('Epoch: {} Loss: {:.4f} Acc: {:.4f}'.format(epoch, val_loss, val_acc))

        #torch.save(model.state_dict(), PATH)   #сохранение весов модели на будущее

        if lr_sched is not None:
            lr_sched.step()

    return train_losses, val_losses, valid_accuracies

Load Model:
    model = TheModelClass(*args, **kwargs)
    model.load_state_dict(torch.load(PATH))
    model.eval()

In [203]:
class MultiLayerPerceptron(nn.Module):

    def __init__(self):
        super(MultiLayerPerceptron, self).__init__()
        # Embedding size, Hyperparameter, Output
        self.conv1 = nn.Conv1d(in_channels=300, out_channels=300, kernel_size=5) #96
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2) #48
        self.conv2 = nn.Conv1d(in_channels=300, out_channels=300, kernel_size=5) #44
        self.conv3 = nn.Conv1d(in_channels=300, out_channels=300, kernel_size=5)
        self.conv4 = nn.Conv1d(in_channels=300, out_channels=300, kernel_size=4)
        self.conv5 = nn.Conv1d(in_channels=300, out_channels=300, kernel_size=2)
        self.fc1 = nn.Linear(300, 30)
        self.fc2 = nn.Linear(30, 3)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = self.pool(F.leaky_relu(self.conv3(x)))
        x = self.pool(F.leaky_relu(self.conv4(x)))
        x = self.pool(F.leaky_relu(self.conv5(x)))
        x = x.view(-1, 300)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        return x

In [204]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [210]:
model = MultiLayerPerceptron().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.9)

info = fit(10, model, criterion, optimizer, train_dataloader, val_dataloader, scheduler)

Epoch::   0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
train_losses = info[0]
plt.figure(figsize=(12, 8))
plt.plot(range(len(train_losses)), train_losses)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

In [ ]:
import sys
sys.path.append('../..')

from sklearn import metrics as multi_metrics

In [ ]:
model.eval()
xb, yb = next(iter(val_dataloader))
probs = model(xb.float())
_, pred = torch.max(probs, axis=-1)

In [ ]:
multi_metrics.f1_score(y_true=yb, y_pred=pred, average='macro')